In [11]:
import cv2
import numpy as np
import math
import time

In [14]:
#cap =cv2.VideoCapture("http://192.168.1.90:8080/video")#http://192.168.1.90:8080
#cap = cv2.VideoCapture("http://10.84.123.48:8080/video")
cap = cv2.VideoCapture("http://10.84.88.194:8080/video")

In [15]:
folder = "C:/Users/Raj/Desktop/Independent Study/calibrate"

In [16]:
counter = 0
while True:
    success, img = cap.read()
    cv2.imshow("im",img)
    
    key = cv2.waitKey(1)    
    if key ==ord("s"):
        counter += 1
        cv2.imwrite(f'{folder}/Image_{time.time()}.jpg',img)
        print(counter)
    elif key ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


In [ ]:
cap.release()

In [7]:
cv2.destroyAllWindows()

In [1]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('*.jpg')
for fname in images:
    img = cv.imread(fname)
    cv.imshow("im",img)
    cv.waitKey(1)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (9,6), None)
    print(ret)
    # If found, add object points, image points (after refining them)
    if ret == True:
        print("a")
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (9,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [7]:
cv.destroyAllWindows()

In [12]:
import numpy as np
import cv2 as cv
import glob



################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (9,6)
frameSize = (1440,1080)
path = r"C:/Users/Raj/Desktop/Independent Study/calibrate/new/*.jpg"


# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
#objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
#objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# objp[:, 0] = np.tile(np.arange(8), 6) * 30
# objp[:, 1] = np.repeat(np.arange(6), 8) * 30
#size_of_chessboard_squares_mm = 27.5
#|objp = objp * size_of_chessboard_squares_mm


# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


#images = glob.glob('*.jpg')
iamges = glob.glob(path)
for image in images:

    img = cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)
    print(ret)
    # If found, add object points, image points (after refining them)
    if ret == True:

        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(100)


cv.destroyAllWindows()




############## CALIBRATION #######################################################

ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)


############## UNDISTORTION #####################################################

img = cv.imread('C:/Users/Raj/Desktop/Independent Study/Image_1685056464.5000937.jpg')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))



# Undistort
dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# # crop the image
# x, y, w, h = roi
# dst = dst[y:y+h, x:x+w]
# cv.imwrite('caliResult1.png', dst)



# Undistort with Remapping
# mapx, mapy = cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
# dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# # crop the image
# x, y, w, h = roi
# dst = dst[y:y+h, x:x+w]
# cv.imwrite('caliResult2.png', dst)




# Reprojection Error
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print( "total error: {}".format(mean_error/len(objpoints)) )
np.save("calibration_matrix",cameraMatrix)
np.save("distortion_coefficients", dist)
np.save("new_calibration_matrix",newCameraMatrix)
print("mat:",cameraMatrix)
print("new:",newCameraMatrix)
print("tvec:",tvecs)
print("rvec:",rvecs)

False
False
False
False
False
False


KeyboardInterrupt: 

In [4]:
cv.destroyAllWindows()

In [ ]:

import numpy as np
import cv2
import sys
import time
import cv2.aruco as aruco

def pose_esitmation(frame, matrix_coefficients, distortion_coefficients):

    '''
    frame - Frame from the video stream
    matrix_coefficients - Intrinsic matrix of the calibrated camera
    distortion_coefficients - Distortion coefficients associated with your camera
    return:-
    frame - The frame with the axis drawn on it
    '''

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.aruco_dict = cv2.aruco.Dictionary_get(aruco.DICT_4X4_50)
    parameters = cv2.aruco.DetectorParameters_create()


    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, cv2.aruco_dict,parameters=parameters,
        cameraMatrix=matrix_coefficients,
        distCoeff=distortion_coefficients)

        # If markers are detected
    if len(corners) > 0:
        for i in range(0, len(ids)):
            # Estimate pose of each marker and return the values rvec and tvec---(different from those of camera coefficients)
            rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.02, matrix_coefficients,
                                                                       distortion_coefficients)
            # Draw a square around the markers
            cv2.aruco.drawDetectedMarkers(frame, corners) 

            # Draw Axis
            cv2.aruco.drawAxis(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)  

    return frame

if __name__ == '__main__':

    #aruco_dict_type = ARUCO_DICT[DICT_4X4_50]
    aruco_type = "DICT_4X4_50"
    arucoDict = aruco.Dictionary_get(ARUCO_DICT[aruco_type])
    # Define the parameters for the marker detection
    arucoParams = aruco.DetectorParameters_create()
    #M = None
    
    distortion_coefficients_path ='C:/Users/Raj/Desktop/Independent Study/calibrate/distortion_coefficients.npy'
    calibration_matrix_path = 'C:/Users/Raj/Desktop/Independent Study/calibrate/new_calibration_matrix.npy'
    
    k = np.load(calibration_matrix_path)
    d = np.load(distortion_coefficients_path)

    #video = cv2.VideoCapture(0)
    video =cv2.VideoCapture("http://192.168.1.90:8080/video")
    time.sleep(2.0)

    while True:
        ret, frame = video.read()

        if not ret:
            break
        
        output = pose_esitmation(frame,ARUCO_DUC  k, d)

        cv2.imshow('Estimated Pose', output)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

In [ ]:
calibration_matrix_path = 'C:/Users/Raj/Desktop/Independent Study/calibrate/calibration_matrix.npy'

# Load the calibration matrix
calibration_matrix = np.load(calibration_matrix_path)

# Use the calibration matrix in your code
# (replace this with your actual code that uses the matrix)
print(calibration_matrix)

In [ ]:
import numpy as np
import cv2.aruco as aruco
import sys
import time
import cv2

ARUCO_DICT = {
	"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
	"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
	"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
	"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
	"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
	"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
	"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
	"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
	"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
	"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
	"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
	"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
	"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
	"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
	"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
	"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
	"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
	"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
	"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
	"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
	"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}

def aruco_display(corners, ids, rejected, image):
    
	if len(corners) > 0:
		
		ids = ids.flatten()
		
		for (markerCorner, markerID) in zip(corners, ids):
			
			corners = markerCorner.reshape((4, 2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners
			
			topRight = (int(topRight[0]), int(topRight[1]))
			bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
			bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
			topLeft = (int(topLeft[0]), int(topLeft[1]))

			cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
			cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
			cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
			cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
			
			cX = int((topLeft[0] + bottomRight[0]) / 2.0)
			cY = int((topLeft[1] + bottomRight[1]) / 2.0)
			cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
			
			cv2.putText(image, str(markerID),(topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
				0.5, (0, 255, 0), 2)
			print("[Inference] ArUco marker ID: {}".format(markerID))
			
	return image



def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(aruco_dict_type)
    parameters = aruco.DetectorParameters_create()


    #corners, ids, rejected_img_points = aruco.detectMarkers(gray, cv2.aruco_dict,parameters=parameters)
    #aruco.drawDetectedMarkers(frame, corners, ids)
    corners, ids, rejected = aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    R_list=[]

    if np.all(ids is not None):  # If there are markers found by detector
        for i in range(0, len(ids)):  # Iterate in markers

        # Estimate pose of each marker and return the values rvec and tvec---different from camera coefficients
            rvec, tvec, markerPoints = aruco.estimatePoseSingleMarkers(corners[i], 2.7, matrix_coefficients,
                                                                   distortion_coefficients)
            (rvec - tvec).any()  # get rid of that nasty numpy value array error
            #print(tvec)


            #aruco.drawDetectedMarkers(frame, corners)  # Draw A square around the markers
            aruco.drawDetectedMarkers(frame, corners, ids)
            cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 3)  # Draw Axis


            R, _ = cv2.Rodrigues(rvec)
            # convert (np.matrix(R).T) matrix to array using np.squeeze(np.asarray()) to get rid off the ValueError: shapes (1,3) and (1,3) not aligned
            R = np.squeeze(np.asarray(np.matrix(R).T))
            R_list.append(R[2])
            #print(len(R_list))


        # Display the resulting frame


    if len(R_list) == 2:


        print('##############')
        angle_radians = np.arccos(np.dot(R_list[0], R_list[1]))
        angle_degrees=angle_radians*180/np.pi
        print(angle_degrees)

#     aruco.drawDetectedMarkers(frame, corners, ids)
#     #return frame
#     if len(corners) > 0:
#         for i in range(0, len(ids)):
           
#             rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.02, matrix_coefficients,
#                                                                        distortion_coefficients)
            
#             cv2.aruco.drawDetectedMarkers(frame, corners) 

#             cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)
#             #result_img = cv2.drawFrameAxes(dst1, mtx, dist, rvec[i, :, :], tvec[i, :, :],0.03)
    
    return frame

# store rz1 and rz2
    
     


    

aruco_type = "DICT_4X4_50"

arucoDict = cv2.aruco.Dictionary_get(ARUCO_DICT[aruco_type])

arucoParams = cv2.aruco.DetectorParameters_create()
#parameters = aruco.DetectorParameters_create()
#M = None
distortion_coefficients_path ='C:/Users/Raj/Desktop/Independent Study/calibrate/distortion_coefficients.npy'
calibration_matrix_path = 'C:/Users/Raj/Desktop/Independent Study/calibrate/new_calibration_matrix.npy'

k = np.load(calibration_matrix_path)
d = np.load(distortion_coefficients_path)

intrinsic_camera = k
distortion = d


cap = cv2.VideoCapture("http://192.168.1.90:8080/video")

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)



while cap.isOpened():
    
    ret, img = cap.read()
    
    output = pose_estimation(img, ARUCO_DICT[aruco_type], intrinsic_camera, distortion)

    cv2.imshow('Estimated Pose', output)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import time
import cv2.aruco as aruco

import time

In [ ]:
aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)

# Define the parameters for the marker detection
parameters = aruco.DetectorParameters_create()
M = None


# A required callback method that goes into the trackbar function.
def nothing(x):
    pass


In [ ]:
cap = cv2.VideoCapture("http://192.168.1.90:8080/video")

In [ ]:
while True:
    ret, frame = cap.read()
    corners, ids, rejected = aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    aruco.drawDetectedMarkers(frame, corners, ids)
    #corners, ids, rejected = aruco.detectMarkers(frame, aruco_
    cv2.imshow("im",frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()
    

In [ ]:
import numpy as np
import cv2.aruco as aruco
import sys
import time
import cv2

ARUCO_DICT = {
	"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
	"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
	"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
	"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
	"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
	"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
	"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
	"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
	"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
	"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
	"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
	"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
	"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
	"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
	"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
	"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
	"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
	"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
	"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
	"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
	"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}

def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(aruco_dict_type)
    parameters = aruco.DetectorParameters_create()
    corners, ids, rejected = aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    R_list=[]
    a =0.0
    b =0.0
    
    #print(ids)

    if np.all(ids is not None):
        if 8 in ids:
            index_a = int(np.where(ids == 8)[0])
        else:
            
            index_a = -1  # Marker ID 7 not found
        if 6 in ids:
            index_b = int(np.where(ids == 6)[0])
        else:
            index_b = -1  # Marker ID 8 not found

        rvec_a, tvec_a, markerPoints_a = aruco.estimatePoseSingleMarkers(corners[index_a], 2.7, matrix_coefficients,distortion_coefficients)
        (rvec_a - tvec_a).any()  # get rid of that nasty numpy value array error
        rvec_b, tvec_b, markerPoints_b = aruco.estimatePoseSingleMarkers(corners[index_b], 2.7, matrix_coefficients,distortion_coefficients)
        (rvec_b - tvec_b).any()  # get rid of that nasty numpy value array error
        xa = tvec_a[0][0][0]
        xb = tvec_b[0][0][0]
        ya = tvec_a[0][0][1]
        yb = tvec_b[0][0][1]
        #print(tvec_a,tvec_b)
        print(f"distance:{xb-xa} and {ya-yb}")
        aruco.drawDetectedMarkers(frame, corners)  # Draw A square around the markers
        aruco.drawDetectedMarkers(frame, corners, ids)
        cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec_a, tvec_a, 2.7)  # Draw Axis
        cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec_b, tvec_b, 2.7)  # Draw Axis
    return frame

aruco_type = "DICT_4X4_50"

arucoDict = cv2.aruco.Dictionary_get(ARUCO_DICT[aruco_type])

arucoParams = cv2.aruco.DetectorParameters_create()
distortion_coefficients_path ='C:/Users/Raj/Desktop/Independent Study/calibrate/distortion_coefficients.npy'
calibration_matrix_path = 'C:/Users/Raj/Desktop/Independent Study/calibrate/new_calibration_matrix.npy'

k = np.load(calibration_matrix_path)
d = np.load(distortion_coefficients_path)

intrinsic_camera = k
distortion = d


cap = cv2.VideoCapture("http://192.168.1.90:8080/video")
while cap.isOpened():
    
    ret, img = cap.read()
    
    output = pose_estimation(img, ARUCO_DICT[aruco_type], intrinsic_camera, distortion)

    cv2.imshow('Estimated Pose', output)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [171]:
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2.aruco as aruco
import sys
import time
import cv2

distortion_coefficients_path ='C:/Users/Raj/Desktop/Independent Study/calibrate/distortion_coefficients.npy'
calibration_matrix_path = 'C:/Users/Raj/Desktop/Independent Study/calibrate/new_calibration_matrix.npy'
matrix_coefficients = np.load(calibration_matrix_path)
distortion_coefficients = np.load(distortion_coefficients_path)

nb_markers=2
cap = cv2.VideoCapture("http://192.168.1.90:8080/video")
def track(matrix_coefficients, distortion_coefficients):
    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
        parameters = aruco.DetectorParameters_create()
        #corners, ids, rejected_img_points = aruco.detectMarkers(gray,aruco_dict,parameters=parameters,cameraMatrix=matrix_coefficients,distCoeff=distortion_coefficients)    
        corners, ids, rejected = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
        R_list=[]
        if np.all(ids is not None): 
            for i in range(0, len(ids)): 
                rvec, tvec, markerPoints = aruco.estimatePoseSingleMarkers(corners[i], 3, matrix_coefficients,distortion_coefficients)
                (rvec - tvec).any() 
                aruco.drawDetectedMarkers(frame, corners)
                #aruco.drawAxis(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)
                cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 3)
                R, _ = cv2.Rodrigues(rvec)
                R = np.squeeze(np.asarray(np.matrix(R).T))
                R_list.append(R[2])
        if len(R_list) == 2:
            print('##############')
            angle_radians = np.arccos(np.dot(R_list[0], R_list[1]))
            angle_degrees=angle_radians*180/np.pi
            print(angle_degrees)
        cv2.imshow('frame', frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            ("Ok")
            break
track(matrix_coefficients, distortion_coefficients)

In [ ]:
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2.aruco as aruco
import sys
import time
import cv2
import math

distortion_coefficients_path ='C:/Users/Raj/Desktop/Independent Study/calibrate/distortion_coefficients.npy'
calibration_matrix_path = 'C:/Users/Raj/Desktop/Independent Study/calibrate/new_calibration_matrix.npy'
matrix_coefficients = np.load(calibration_matrix_path)
distortion_coefficients = np.load(distortion_coefficients_path)
nb_markers=2
#cap = cv2.VideoCapture("http://192.168.1.90:8080/video")
cap = cv2.VideoCapture("http://10.84.88.194:8080/video")
#10.84.88.194

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (480 * 2, 270 * 2))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
    parameters = aruco.DetectorParameters_create()
    #corners, ids, rejected_img_points = aruco.detectMarkers(gray,aruco_dict,parameters=parameters,cameraMatrix=matrix_coefficients,distCoeff=distortion_coefficients)    
    corners, ids, rejected = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    print(ids)
    if np.all(ids is not None):
        
        
        if 8 in ids:
            index_a = int(np.where(ids == 8)[0])
        else:
            index_a = -1  # Marker ID 7 not found
        if 7 in ids:
            index_b = int(np.where(ids == 7)[0])
        else:
            index_b = -1  # Marker ID 8 not foundd
        rvec_6, tvec_6, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[index_b], 3, matrix_coefficients, distortion_coefficients)
        rvec_7, tvec_7, markerPoints_7 = cv2.aruco.estimatePoseSingleMarkers(corners[index_a], 3, matrix_coefficients, distortion_coefficients)
        #print(tvec_3)
        #print(tvec_7)
        #R1, _ = cv2.Rodrigues(rvec_3)
        #R2, _ = cv2.Rodrigues(rvec_7)
        rmat_3, jacobian_3 = cv2.Rodrigues(rvec_6)
        isRotationMatrix(rmat_3)
        a_6 = rotationMatrixToEulerAngles(rmat_3)
        #print(a_3[2])
        rmat_7, jacobian_7 = cv2.Rodrigues(rvec_7)
        isRotationMatrix(rmat_7)
        a_7 = rotationMatrixToEulerAngles(rmat_7)
        #print(a_7[2])
        #angle_radians = np.arccos(np.dot(a_3[2],  a_7[2]))
        #print(f"angle:{(360.0 -abs(a_7[2] - a_6[2]))} and {(a_6[2] - a_7[2])}")
    # Extract the third columns of the rotation matrices
#         rz1 = R1[:, 2]
#         rz2 = R2[:, 2]
#         #rz1  = rvec_3[0][0][2]
#         #rz2 = rvec_7[0][0][2]
#         angle_radians = np.arccos(np.dot(rz1, rz2))
#         angle_degrees=angle_radians*180/np.pi
#         print("angle: ",angle_degrees)
        a = tvec_6[0][0][0]
        b = tvec_7[0][0][0]
        a_ = tvec_6[0][0][1]
        b_ = tvec_7[0][0][1]
        print(f"distance:{(a_-b_)} and{(a-b)} ")
        
#         theta_deg = float(input("Enter the rotation angle of the robot in degrees: "))
#         x_obj_world = float(input("Enter the x-coordinate of the object in the world frame: "))
#         y_obj_world = float(input("Enter the y-coordinate of the object in the world frame: "))
#         x_robot_world = float(input("Enter the x-coordinate of the robot in the world frame: "))
#         y_robot_world = float(input("Enter the y-coordinate of the robot in the world frame: "))
#         theta_rad = math.radians(theta_deg)

#         # Calculate relative position in the world frame
#         x_rel_world = x_obj_world - x_robot_world
#         y_rel_world = y_obj_world - y_robot_world

#         # Apply reverse rotation transformation to obtain coordinates in the robot's frame
#         x_rel_robot = x_rel_world * math.cos(theta_rad) + y_rel_world * math.sin(theta_rad)
#         y_rel_robot = -x_rel_world * math.sin(theta_rad) + y_rel_world * math.cos(theta_rad)

        # Optionally, apply translation if needed
#         x_obj_robot = x_rel_robot
#         y_obj_robot = y_rel_robot
        #print("angle: ",rz1 -rz2)
    
        aruco.drawDetectedMarkers(frame, corners)  # Draw A square around the markers
        aruco.drawDetectedMarkers(frame, corners, ids)
        cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec_6, tvec_6, 30)  # Draw Axis
        cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec_7, tvec_7, 30)  # Draw Axis
    cv2.imshow('frame', frame)
#     #print(frame.shape)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        ("Ok")
        break
cv2.destroyAllWindows()

[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712

[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712

[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712577486 
[[8]
 [7]]
distance:-11.428005166557796 and146.43894712

[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 an

[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[8]
 [7]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[8]
 [7]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.20801120170535 and86.361695722153 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and8

[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 an

[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 an

[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.27988744387375 and85.79430948753458 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 and86.83298525667749 
[[7]
 [8]]
distance:73.95061084532716 an

[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 
[[7]
 [8]]
distance:72.49993654403721 and85.33786757718389 


In [ ]:
cv2.destroyAllWindows()

In [2]:
def isRotationMatrix(R) : 
    Rt = np.transpose(R) 
    shouldBeIdentity = np.dot(Rt, R) 
    I = np.identity(3, dtype = R.dtype) 
    n = np.linalg.norm(I - shouldBeIdentity) 
    return n < 1e-6

In [3]:
def rotationMatrixToEulerAngles(R): 
    assert(isRotationMatrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] + R[1,0] * R[1,0])

    singular = sy < 1e-6

    if not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
        # z = math.atan2(R[0,1], -R[1,1])
    else:
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.array([x, y, z]) * (180/np.pi) # move 

In [4]:
from uarm.wrapper import SwiftAPI
#swift = SwiftAPI()
import cv2, queue, threading, time
import cv2
import numpy as np
import time
import cv2.aruco as aruco
from uarm.wrapper import SwiftAPI
import time
import queue, threading, time
import math

aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
parameters = aruco.DetectorParameters_create()
distortion_coefficients_path ='C:/Users/Raj/Desktop/Independent Study/calibrate/distortion_coefficients.npy'
calibration_matrix_path = 'C:/Users/Raj/Desktop/Independent Study/calibrate/new_calibration_matrix.npy'

matrix_coefficients = np.load(calibration_matrix_path)
distortion_coefficients = np.load(distortion_coefficients_path)

M = None
wname = "Trackbars"
cv2.namedWindow(wname)
wname1 = "image"
cv2.namedWindow(wname1)

l_h = 0
l_s = 0
l_v = 0
u_h = 179
u_s = 255
u_v = 255
hsv = None

def move_robot(u, v):
    x = round(u, 2)
    y = 88+round(v, 2)
    swift.set_position(x=x, y=y, z=100, speed=10000)
    time.sleep(1.0)
    swift.set_position(x=x, y=y, z=40, speed=10000)
    time.sleep(1.0)
    swift.set_pump(on=True)
    time.sleep(2)
    swift.set_position(x=x, y=x, z=100, speed=10000)
    time.sleep(1)
    swift.set_position(x=0, y=-100, z=100, speed=10000)
    time.sleep(1.0)
    swift.set_position(x=0, y=-100, z=70, speed=10000)
    time.sleep(1.0)
    swift.set_pump(on=False)
    time.sleep(1.0)
    swift.set_position(x=0, y=0, z=100, speed=10000)

class VideoCapture:
    def __init__(self, name):
        self.cap = cv2.VideoCapture(name)
        self.q = queue.Queue()
        t = threading.Thread(target=self._reader)
        t.daemon = True
        t.start()

    def _reader(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            if not self.q.empty():
                try:
                    self.q.get_nowait()
                except queue.Empty:
                    pass
            self.q.put(frame)

    def read(self):
        return self.q.get()
    def release(self):
        return self.cap.release()

def isRotationMatrix(R) : 
    Rt = np.transpose(R) 
    shouldBeIdentity = np.dot(Rt, R) 
    I = np.identity(3, dtype = R.dtype) 
    n = np.linalg.norm(I - shouldBeIdentity) 
    return n < 1e-6

def rotationMatrixToEulerAngles(R): 
    assert(isRotationMatrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] + R[1,0] * R[1,0])

    singular = sy < 1e-6

    if not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
        # z = math.atan2(R[0,1], -R[1,1])
    else:
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.array([x, y, z]) * (180/np.pi) # move 

def pixel_mm(u,v):
    sccale_factor = (12*25.4)/500
    x_cord = u*sccale_factor
    y_coord = v *sccale_factor
    return x_cord,y_coord


def onclick(event, x, y, flags, param):
    global u_s, u_v, u_h, l_h, l_s, l_v, hsv
    if event == cv2.EVENT_LBUTTONDOWN:
        print("x: %d, y: %d" % (x, y))
        l_h = max(hsv[y, x][0] - 10, 0)
        l_s = max(hsv[y, x][1] - 40, 0)
        l_v = max(hsv[y, x][2] - 40, 0)
        u_h = min(hsv[y, x][0] + 10, 179)
        u_s = min(hsv[y, x][1] + 40, 255)
        u_v = min(hsv[y, x][2] + 40, 255)
    

rectangle = None
cv2.setMouseCallback(wname1, onclick)

#swift.connect()
#cap = VideoCapture("http://10.84.18.188:8080/video")
#cap = VideoCapture("http://192.168.1.90:8080/video")
cap = VideoCapture("http://10.84.88.194:8080/video")
while True:
    frame = cap.read()
    frame = cv2.resize(frame, (480 * 2, 270 * 2))
    frame_new = frame
    corners, ids, rejected = aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    if np.all(ids is not None):
        if 7 in ids:
            index_a = int(np.where(ids == 7)[0])
        else:
            index_a = -1  # Marker ID 7 not found
        if 8 in ids:
            index_b = int(np.where(ids == 8)[0])
        else:
            index_b = -1  # Marker ID 8 not found

        rvec_a, tvec_a, markerPoints_a = aruco.estimatePoseSingleMarkers(corners[index_a], 3, matrix_coefficients,distortion_coefficients)
        (rvec_a - tvec_a).any()  # get rid of that nasty numpy value array error
        rvec_b, tvec_b, markerPoints_b = aruco.estimatePoseSingleMarkers(corners[index_b], 3, matrix_coefficients,distortion_coefficients)
        (rvec_b - tvec_b).any()  # get rid of that nasty numpy value array error
        rmat_3, jacobian_3 = cv2.Rodrigues(rvec_b)
        isRotationMatrix(rmat_3)
        a_3 = rotationMatrixToEulerAngles(rmat_3)
        #print(a_3[2])
        rmat_7, jacobian_7 = cv2.Rodrigues(rvec_a)
        isRotationMatrix(rmat_7)
        a_7 = rotationMatrixToEulerAngles(rmat_7)
        #print(f"angle:{(360.0 -abs(a_7[2] - a_3[2]))} and {(a_3[2] - a_7[2])}")
        a = tvec_a[0][0][0]
        b = tvec_b[0][0][0]
        a_ = tvec_a[0][0][1]
        b_ = tvec_b[0][0][1]
        ya = (b-a)
        xa =(a_-b_)
        
        print(f"distance:{ya} and {xa}")
        aruco.drawDetectedMarkers(frame, corners)  # Draw A square around the markers
        aruco.drawDetectedMarkers(frame, corners, ids)
        cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec_a, tvec_a, 3)  # Draw Axis
        cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec_b, tvec_b, 3)  # Draw Axis
    
    #rvec, tvec, markerPoints = aruco.estimatePoseSingleMarkers(corners[i], 3, matrix_coefficients,distortion_coefficients)
    output_coords = np.float32([[0, 0], [500, 0], [500, 300], [0, 300]])
    if len(corners) == 4 and M is None:
        src_pts = np.float32([corners[np.where(ids == 3)[0][0]][0][0], corners[np.where(ids == 2)[0][0]][0][1],
                          corners[np.where(ids == 0)[0][0]][0][2], corners[np.where(ids == 1)[0][0]][0][3]])
        dst_pts = np.float32([[0, 0], [0, 500], [500, 500], [500, 0]])
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    if M is not None:
        warped = cv2.warpPerspective(frame, M, (500, 500))
        frame = warped
    aruco.drawDetectedMarkers(frame, corners, ids)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    mask = cv2.inRange(hsv, lower_range, upper_range)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ball_count = 0
    cordinates = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 200 and area < 10000:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            radius = int(radius)
            moments = cv2.moments(contour)
            cv2.circle(frame, center, radius, (0, 255, 0), 2)
            centroid_x = int(moments['m10'] / moments['m00'])
            centroid_y = int(moments['m01'] / moments['m00'])
            cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 255), -1)
            ball_count += 1
            cordinates.append(center)
            if cordinates is not None:
                for i in cordinates:
                    u, v = i
                    x, y = pixel_mm(u, v)
                    print(x,y)
                    x = x-xa
                    y =y 
                    print("length:", len(cordinates))
                    print("moving to:", (x, y))
                    move_robot(x,y)
                cordinates = []

    cv2.imshow('Trackbars', mask)
    cv2.imshow("test",frame_new)
    frame = cv2.putText(frame, "X", (250, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 3)
    cv2.imshow(wname1, frame)
    if chr(cv2.waitKey(1)&255) == 'q':
        break

cv2.destroyAllWindows()
cap.release()
swift.reset()


distance:-4.2444272161114895 and 2.435879207443481
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.2444272161114895 and 2.435879207443481
distance:-3.8837640789297616 and 2.2336620640700744
distance:-4.222884973621053 and 2.4283542927366923
distance:-4.222884973621053 and 2.4283542927366923
distance:-4.222884973621053 and 2.4283542927366923
distance:-4.222884973621053 and 2.4283542927366923
distance:-4.222884973621053 and 2.4283542927366923
distance:-4.2444272161114895 and 2.435879207443481
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.862221836439325 and 2.2261371493632858
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.22439100978

distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.8837640789297616 and 2.2336620640700744
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.222884973621053 and 2.4283542927366923
distance:-4.222884973621053 and 2.4283542927366923
distance:-4.222884973621053 and 2.4283542927366923
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.222884973621053 and 2.4283542927366923
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.2444272161114895 and 2.435879207443481
distance:-4.24442721611148

distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.5997703578486373 and 2.1113814503726758
distance:-2.9091167990600546 and 1.6492353226450476
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.5997703578486373 and 2.1113814503726758
distance:-4.258484124649286 and 2.4867649215601553
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.8837640789297616 and 2.2336620640700744
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.5997703578486373 and 2.1113814503726758
distance:-3.2

distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-4.258484124649286 and 2.4867649215601553
distance:-4.258484124649286 and 2.4867649215601553
distance:-4.258484124649286 and 2.4867649215601553
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.943209913922061 and 2.268130618644456
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.943209913922061 and 2.268130618644456
distance:-3.943209913922061 and 2.268130618644456
distance:-3.943209913922061 and 2.268130618644456
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.28449614712

distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.2844961471214127 and 1.8927471474569764
distance:-3.

...

distance:-12.300591098037966 and -9.498394962749646
distance:-12.300591098037966 and -9.498394962749646
distance:-12.300591098037966 and -9.498394962749646
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.540287530858656 and -9.308395736533818
distance:-12.300591098037966 and -9.498394962749646
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.300591098037966 and -9.498394962749646
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806

distance:-12.328587255806646 and -9.42470180725185
distance:-12.304928303236451 and -9.433824980175622
distance:-12.304928303236451 and -9.433824980175622
distance:-12.304928303236451 and -9.433824980175622
distance:-12.304928303236451 and -9.433824980175622
distance:-12.328587255806646 and -9.42470180725185
distance:-12.304928303236451 and -9.433824980175622
distance:-12.304928303236451 and -9.433824980175622
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.540287530858656 and -9.308395736533818
distance:-12.285928598751308 and -9.47116392495413
distance:-12.146632089164477 and -9.682817896175873
distance:-12.304928303236451 and -9.433824980175622
distance:-12.328587255806646 and -9.42470180725185
distance:-12.304928303236451 and -9.433824980175622
distance:-12.32858725

distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.304928303236451 and -9.433824980175622
distance:-12.304928303236451 and -9.433824980175622
distance:-12.328587255806646 and -9.42470180725185
distance:-12.328587255806646 and -9.42470180725185
distance:-12.304928303236451 and -9.433824980175622
distance:-12.304928303236451 and -9.433824980175622
distance:-12.304928303236451 and -9.433824980175622
distance:-12.304928303236451 and -9.433824980175622
distance:-12.328587255806646 and -9.42470180725185
distance:-12.304928303236451 and -9.433824980175622
distance:-12.30492830323

distance:-12.683914637958495 and -9.148709392491668
distance:-12.512291373089976 and -9.382088892031614
distance:-12.406453155463321 and -9.336595725069166
distance:-12.300591098037966 and -9.498394962749646
distance:-12.549860714325522 and -9.36318592826403
distance:-12.406453155463321 and -9.336595725069166
distance:-12.285928598751308 and -9.47116392495413
distance:-12.419781712035185 and -9.342605960316362
distance:-12.276932145467772 and -9.507518135673418
distance:-12.274301385476608 and -9.480873626958982
distance:-12.285928598751308 and -9.47116392495413
distance:-12.406453155463321 and -9.336595725069166
distance:-12.461009213040676 and -9.25119456733037
distance:-12.304928303236451 and -9.433824980175622
distance:-12.435892011669857 and -9.309737370314767
distance:-12.287629942048472 and -9.486883862206177
distance:-12.422563455097993 and -9.303727135067572
distance:-12.109644580022419 and -9.68593083385245
distance:-12.304928303236451 and -9.433824980175622
distance:-12.6872

NameError: name 'swift' is not defined

In [11]:
cv2.destroyAllWindows()